In [3]:
from __future__ import print_function, division

import os
import time
import copy
import io
import PIL
from PIL import Image
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.io import read_image
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import requests

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
# from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from torchvision.models import resnet50

import openslide
import skimage
import random

plt.ion()   # interactive mode

In [4]:
os.chdir("/project/GutIntelligenceLab/bwl3xy/data/inova_vs_cinc_crohns")
os.getcwd()

'/project/GutIntelligenceLab/bwl3xy/data/inova_vs_cinc_crohns'

In [6]:
image_val_df = pd.read_csv("inova_vs_cinc_val.csv", header = None, names = ["image", "class", "label"])
val_inova_df = image_val_df[image_val_df['class'] == 0]
val_cinc_df = image_val_df[image_val_df['class'] == 1]

image_train_df = pd.read_csv("inova_vs_cinc_train.csv", header = None, names = ["image", "class", "label"])
train_inova_df = image_train_df[image_train_df['class'] == 0]
train_cinc_df = image_train_df[image_train_df['class'] == 1]

In [136]:
path_root = "/project/GutIntelligenceLab/bwl3xy/data/inova_vs_cinc_crohns"
val_image_path = "/inova/" + val_inova_df.iloc[random.randint(0, len(val_inova_df))].image
train_image_path = "/inova/" + train_inova_df.iloc[random.randint(0, len(train_inova_df))].image
val_image = Image.open(path_root + val_image_path)
train_image = Image.open(path_root + train_image_path)

In [137]:
rgb_val_image = np.float32(np.array(val_image))/255
rgb_train_image = np.float32(np.array(train_image))/255

In [138]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [139]:
input_tensor = preprocess_image(rgb_train_image, mean, std)

In [140]:
model = torch.load("/project/GutIntelligenceLab/bwl3xy/models/inova_vs_cinc_crohns.model", map_location ='cpu')
model.eval()
target_layers = [model.layer4[-1]]

In [141]:
cam = GradCAM(model = model, target_layers = target_layers)

In [142]:
targets = None

In [143]:
grayscale_cam = cam(input_tensor = input_tensor)#, targets = None)
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_train_image, grayscale_cam, use_rgb = True)

In [144]:
viz = Image.fromarray(visualization)
viz.save("inova_crohns_gradcam6.jpg")